<a href="https://colab.research.google.com/github/MukeshSiddh/Companion-Arbiter-PUF/blob/main/Mukesh_CS771_eval_assn1_23_24_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
#from submit import my_map
#from submit import my_fit
import time as tm
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import scipy
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV


In [ ]:

import numpy as np

def my_map(X):
    C = 2*X - 1
    D = np.flip(C, axis=1)
    X = np.cumprod(D, axis=1)
    X = np.flip(X, axis=1)
    ones_column = np.ones((X.shape[0], 1))
    X = np.hstack((X, ones_column))

    num_features = X.shape[1]
    num_new_features = (num_features * (num_features - 1)) // 2

    X_final = np.empty((X.shape[0], num_new_features))
    index = 0
    for i in range(num_features):
        for j in range(i + 1, num_features):
            X_final[:, index] = X[:, i] * X[:, j]
            index += 1

    print(X_final.shape)
    return X_final





In [ ]:
# Z_tst = np.loadtxt( "secret_test.dat" )




# def my_map2(X):
#     C = 2*X - 1
#     D = np.flip(C, axis=1)
#     X = np.cumprod(D, axis=1)
#     X = np.flip(X, axis=1)
#     X_final = X

#     for i in range(X.shape[1]):
#         # Multiply the ith column with the matrix formed from i+1th column to the last column
#         col_i = np.expand_dims(X[:, i], axis=1)
#         submatrix = X[:, i+1:]
#         new_features =col_i*submatrix
#         X_final = np.hstack((X_final, new_features))

#     print(X_final.shape)
#     return X_final



# import numpy as np

# def my_map2(X):
#     C = 2*X - 1
#     D = np.flip(C, axis=1)
#     X = np.cumprod(D, axis=1)
#     X = np.flip(X, axis=1)
#     ones_column = np.ones((X.shape[0], 1))
#     X = np.hstack((X, ones_column))

#     num_features = X.shape[1]
#     num_new_features = (num_features * (num_features - 1)) // 2

#     X_final = np.empty((X.shape[0], num_new_features))
#     index = 0
#     for i in range(num_features):
#         for j in range(i + 1, num_features):
#             X_final[:, index] = X[:, i] * X[:, j]
#             index += 1

#     print(X_final.shape)
#     return X_final


# t_map = 0
# tic = tm.perf_counter()
# feat = my_map2( Z_tst[:, :-1] )
# toc = tm.perf_counter()
# t_map += toc - tic
# print(t_map)

In [ ]:
# def my_fit(X_train, y_train):

#     # Map the challenges to feature vectors
#     X_train_mapped = my_map(X_train)

#     # Initialize Logistic Regression model
#     clf = LogisticRegression(max_iter=1000)

#     # Fit the model
#     clf.fit(X_train_mapped, y_train)

#     # Extract coefficients (weights) and intercept (bias)
#     w = clf.coef_.flatten()  # Coefficients of the linear model
#     b = clf.intercept_[0]    # Intercept of the linear model

#     return w, b
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


# def my_fit(X_train, y_train):
#     # Map the challenges to feature vectors
#     X_train_mapped = my_map(X_train)
#     # Scale the data
#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train_mapped)
#     # Initialize Logistic Regression model
#     clf = LogisticRegression(max_iter=10000)

#     # Define hyperparameters to tune
#     parameters = {
#         'C': [10, 100,1000],  # Regularization parameter
#         #'penalty': ['l1', 'l2']  # Regularization penalty
#     }

#     # Perform grid search with cross-validation
#     grid_search = GridSearchCV(clf, parameters, cv=5, n_jobs=-1)
#     grid_search.fit(X_train_mapped, y_train)

#     # Get the best model
#     best_clf = grid_search.best_estimator_
#     print("Best Parameters of the Classifier:")
#     print(grid_search.best_params_)
#     # Fit the best model
#     best_clf.fit(X_train_mapped, y_train)

#     # Extract coefficients (weights) and intercept (bias)
#     w = best_clf.coef_.flatten()  # Coefficients of the linear model
#     b = best_clf.intercept_[0]    # Intercept of the linear model

#     return w, b

def my_fit(X_train, y_train):
    # Map the challenges to feature vectors
    X_train_mapped = my_map(X_train)

    # Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_mapped)

    # Initialize Logistic Regression model with C=100
    clf = LogisticRegression(C=100, max_iter=10000)

    # Fit the model
    clf.fit(X_train_mapped, y_train)

    # Extract coefficients (weights) and intercept (bias)
    w = clf.coef_.flatten()  # Coefficients of the linear model
    b = clf.intercept_[0]    # Intercept of the linear model

    return w, b


In [ ]:
Z_trn = np.loadtxt( "secret_train.dat" )
Z_tst = np.loadtxt( "secret_test.dat" )

n_trials = 5

d_size = 0
t_train = 0
t_map = 0
acc = 0

In [ ]:
for t in range( n_trials ):
	tic = tm.perf_counter()
	w, b = my_fit( Z_trn[:, :-1], Z_trn[:,-1] )
	toc = tm.perf_counter()
	t_train += toc - tic

	d_size += w.shape[0]

	tic = tm.perf_counter()
	feat = my_map( Z_tst[:, :-1] )
	toc = tm.perf_counter()
	t_map += toc - tic

	scores = feat.dot( w ) + b
	pred = np.zeros_like( scores )
	pred[scores > 0] = 1
	acc += np.average( Z_tst[ :, -1 ] == pred )

(40000, 528)
(10000, 528)
(40000, 528)
(10000, 528)
(40000, 528)
(10000, 528)
(40000, 528)
(10000, 528)
(40000, 528)
(10000, 528)


In [ ]:
d_size /= n_trials
t_train /= n_trials
t_map /= n_trials
acc /= n_trials

print( d_size, t_train, t_map,  acc )


528.0 2.1085357193999696 0.08965013539991559 0.9930999999999999


In [ ]:
feat.shape

(10000, 528)